In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:
df_denmark_2021 = pd.read_csv('denmark/DK_2021_hourly.csv')
df_denmark_2022 = pd.read_csv('denmark/DK_2022_hourly.csv')
df_denmark_2023 = pd.read_csv('denmark/DK_2023_hourly.csv')
df_denmark_2024 = pd.read_csv('denmark/DK_2024_hourly.csv')

df_denmark_2021['Datetime (UTC)'] = pd.to_datetime(df_denmark_2021['Datetime (UTC)'])
df_denmark_2022['Datetime (UTC)'] = pd.to_datetime(df_denmark_2022['Datetime (UTC)'])
df_denmark_2023['Datetime (UTC)'] = pd.to_datetime(df_denmark_2023['Datetime (UTC)'])
df_denmark_2024['Datetime (UTC)'] = pd.to_datetime(df_denmark_2024['Datetime (UTC)'])

In [ ]:
df_finland_2021 = pd.read_csv('finland/FI_2021_hourly.csv')
df_finland_2022 = pd.read_csv('finland/FI_2022_hourly.csv')
df_finland_2023 = pd.read_csv('finland/FI_2023_hourly.csv')
df_finland_2024 = pd.read_csv('finland/FI_2024_hourly.csv')

df_finland_2021['Datetime (UTC)'] = pd.to_datetime(df_finland_2021['Datetime (UTC)'])
df_finland_2022['Datetime (UTC)'] = pd.to_datetime(df_finland_2022['Datetime (UTC)'])
df_finland_2023['Datetime (UTC)'] = pd.to_datetime(df_finland_2023['Datetime (UTC)'])
df_finland_2024['Datetime (UTC)'] = pd.to_datetime(df_finland_2024['Datetime (UTC)'])

In [ ]:
df_iceland_2021 = pd.read_csv('iceland/IS_2021_hourly.csv')
df_iceland_2022 = pd.read_csv('iceland/IS_2022_hourly.csv')
df_iceland_2023 = pd.read_csv('iceland/IS_2023_hourly.csv')
df_iceland_2024 = pd.read_csv('iceland/IS_2024_hourly.csv')

df_iceland_2021['Datetime (UTC)'] = pd.to_datetime(df_iceland_2021['Datetime (UTC)'])
df_iceland_2022['Datetime (UTC)'] = pd.to_datetime(df_iceland_2022['Datetime (UTC)'])
df_iceland_2023['Datetime (UTC)'] = pd.to_datetime(df_iceland_2023['Datetime (UTC)'])
df_iceland_2024['Datetime (UTC)'] = pd.to_datetime(df_iceland_2024['Datetime (UTC)'])

In [ ]:
df_norway_2021 = pd.read_csv('norway/NO_2021_hourly.csv')
df_norway_2022 = pd.read_csv('norway/NO_2022_hourly.csv')
df_norway_2023 = pd.read_csv('norway/NO_2023_hourly.csv')
df_norway_2024 = pd.read_csv('norway/NO_2024_hourly.csv')

df_norway_2021['Datetime (UTC)'] = pd.to_datetime(df_norway_2021['Datetime (UTC)'])
df_norway_2022['Datetime (UTC)'] = pd.to_datetime(df_norway_2022['Datetime (UTC)'])
df_norway_2023['Datetime (UTC)'] = pd.to_datetime(df_norway_2023['Datetime (UTC)'])
df_norway_2024['Datetime (UTC)'] = pd.to_datetime(df_norway_2024['Datetime (UTC)'])

In [ ]:
df_sweden_2021 = pd.read_csv('sweden/SE_2021_hourly.csv')
df_sweden_2022 = pd.read_csv('sweden/SE_2022_hourly.csv')
df_sweden_2023 = pd.read_csv('sweden/SE_2023_hourly.csv')
df_sweden_2024 = pd.read_csv('sweden/SE_2024_hourly.csv')

df_sweden_2021['Datetime (UTC)'] = pd.to_datetime(df_sweden_2021['Datetime (UTC)'])
df_sweden_2022['Datetime (UTC)'] = pd.to_datetime(df_sweden_2022['Datetime (UTC)'])
df_sweden_2023['Datetime (UTC)'] = pd.to_datetime(df_sweden_2023['Datetime (UTC)'])
df_sweden_2024['Datetime (UTC)'] = pd.to_datetime(df_sweden_2024['Datetime (UTC)'])

In [ ]:
dataframes = {
    'denmark': {
        2021: df_denmark_2021,
        2022: df_denmark_2022,
        2023: df_denmark_2023,
        2024: df_denmark_2024,
    },
    'finland': {
        2021: df_finland_2021,
        2022: df_finland_2022,
        2023: df_finland_2023,
        2024: df_finland_2024,
    },
    'iceland': {
        2021: df_iceland_2021,
        2022: df_iceland_2022,
        2023: df_iceland_2023,
        2024: df_iceland_2024,
    },
    'norway': {
        2021: df_norway_2021,
        2022: df_norway_2022,
        2023: df_norway_2023,
        2024: df_norway_2024,
    },
    'sweden': {
        2021: df_sweden_2021,
        2022: df_sweden_2022,
        2023: df_sweden_2023,
        2024: df_sweden_2024,
    }
}

for country, years in dataframes.items():
    for year, df in years.items():
        print(f'{country} - {year}: {df.shape}')


In [ ]:
db_user = 'postgres'
db_password = '123456'
db_host = 'localhost'
db_port = '5432'
db_name = 'nordic_co2'

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


with engine.connect() as conn:
    for country in dataframes.keys():
        conn.execute(text(f'CREATE SCHEMA IF NOT EXISTS {country};'))
    conn.commit()

for country, years in dataframes.items():
    for year, df in years.items():
        table_name = f'year_{year}_hourly'

        df = df.rename(columns={
            "Datetime (UTC)": "datetime_utc",
            "Country": "country",
            "Zone Name": "zone_name",
            "Zone Id": "zone_id",
            "Carbon Intensity gCO₂eq/kWh (direct)": "carbon_intensity_direct",
            "Carbon Intensity gCO₂eq/kWh (LCA)": "carbon_intensity_lca",
            "Low Carbon Percentage": "low_carbon_percentage",
            "Renewable Percentage": "renewable_percentage",
            "Data Source": "data_source",
            "Data Estimated": "data_estimated",
            "Data Estimation Method": "data_estimation_method"
        })
        
        df.to_sql(table_name, con=engine, schema=country, if_exists='replace', index=False)
        print(f'Tabela "{country}.{table_name}" carregada com sucesso!')
